In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import os

In [3]:
os.chdir('D:\Leon Projects\Python\Vedenie v ML\Ved ml_week7\data')

## Загрузка и предобработка данных

In [4]:
features_data = pd.read_csv('features.csv',index_col='match_id')

In [5]:
features_data.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [12]:
#Признаки которые нужно удалить
cols_for_del=['duration','radiant_win','tower_status_radiant', 'tower_status_dire',
       'barracks_status_radiant', 'barracks_status_dire']

In [13]:
X = features_data.drop(cols_for_del, axis=1)
y = features_data['radiant_win'] # Вектор целевых переменных

In [12]:
#Есть ли пропущенные данные
X.shape[0] - X[X.columns[X.count() < X.shape[0]]].count()

first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
first_blood_player2            43987
radiant_bottle_time            15691
radiant_courier_time             692
radiant_flying_courier_time    27479
radiant_first_ward_time         1836
dire_bottle_time               16143
dire_courier_time                676
dire_flying_courier_time       26098
dire_first_ward_time            1826
dtype: int64

In [26]:
# Замена пропусков на 0
X.fillna(0, inplace=True)

## Градиентный бустинг

In [18]:
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier

In [19]:
kf = KFold(X.shape[0],shuffle=True,n_folds=5)

In [16]:
n_trees = [10,20,30]

In [20]:
import datetime

In [25]:
times =[]
crosses=[]
for n in n_trees:
    estimator = GradientBoostingClassifier(n_estimators=n)
    start = datetime.datetime.now()
    cross = cross_val_score(estimator,X,y,cv=kf, scoring='roc_auc')
    end = datetime.datetime.now()
    crosses.append(cross)
    times.append(end - start)
    print 'Кол-во деревьев - %s ; время обучения - %s ; Оценка - %s' % (n,times[-1],crosses[-1].mean())

Кол-во деревьев - 10 ; время обучения - 0:07:16.104000 ; Оценка - 0.664540295947
Кол-во деревьев - 20 ; время обучения - 0:16:35.374000 ; Оценка - 0.681785955896
Кол-во деревьев - 30 ; время обучения - 0:20:38.328000 ; Оценка - 0.689285167257


### Отсчет по градиентному бустингу

1. Признаки, которые имели пропуски в данных:
        first_blood_time             
        first_blood_team               
        first_blood_player1            
        first_blood_player2            
        radiant_bottle_time            
        radiant_courier_time             
        radiant_flying_courier_time    
        radiant_first_ward_time         
        dire_bottle_time               
        dire_courier_time                
        dire_flying_courier_time    
        dire_first_ward_time
  Отсутствие данных по этим признакам говорит о том, что за первые 5 минут не произошло событие.  
  На пример: <b>first_blood_time</b> - не был убит ни один герой;  <b>radiant_bottle_time</b> - светлая сторона не собрала ни одной бутылки 
<br><br>
2. Столбец целевых переменных - <b>radiant_win</b>... 1 - Если светлые победили, иначе 0
<br><br>
3. При 30 базовых алгоримах, оценка кросс валидации составляет примерно <b>0.689</b>. Время обучение увеличилось по cрасвнению с меньшим кол-вом алгоритмов(в моем случае около 20 минут из-за довольно древнего железа). Это время можно было бы уменьшить используя стахостический метод градиентного бустинга. В таком случае сходимость может достигнуть быстрее, не используя всю выборку.  
<br>
4. Видно, что при увеличении кол-ва алгоримов, качество улучшается, но не стоит увлекаться, так как на слишком большом кол-ве, градиентный бустинг будет переобучаться  

## Логистическая регрессия

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV

In [43]:
scaler = StandardScaler()

In [15]:
C = {'C' : [0.01, 0.03, 0.1, 0.3, 1, 3]}

#### 1 Задание

In [44]:
X_scaled = scaler.fit_transform(X)

In [21]:
log_reg = LogisticRegression(penalty='l2')

In [22]:
grid_search = GridSearchCV(log_reg,param_grid=C, scoring='roc_auc', cv=kf,verbose=5)

In [64]:
grid_all_features = grid_search.fit(X_scaled,y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] C=0.01 ..........................................................
[CV] ................................. C=0.01, score=0.716929 -  14.4s
[CV] C=0.01 ..........................................................
[CV] ................................. C=0.01, score=0.715285 -  13.3s
[CV] C=0.01 ..........................................................
[CV] ................................. C=0.01, score=0.717703 -  13.3s
[CV] C=0.01 ..........................................................
[CV] ................................. C=0.01, score=0.715575 -  13.2s
[CV] C=0.01 ..........................................................
[CV] ................................. C=0.01, score=0.716399 -  14.1s
[CV] C=0.03 ..........................................................
[CV] ................................. C=0.03, score=0.716952 -  14.7s
[CV] C=0.03 ..........................................................
[CV] ............

[Parallel(n_jobs=1)]: Done  17 tasks       | elapsed:  4.0min


[CV] .................................. C=0.3, score=0.717675 -  14.2s
[CV] C=0.3 ...........................................................
[CV] .................................. C=0.3, score=0.715450 -  14.7s
[CV] C=0.3 ...........................................................
[CV] .................................. C=0.3, score=0.716368 -  14.8s
[CV] C=1 .............................................................
[CV] .................................... C=1, score=0.716965 -  15.6s
[CV] C=1 .............................................................
[CV] .................................... C=1, score=0.715254 -  13.8s
[CV] C=1 .............................................................
[CV] .................................... C=1, score=0.717674 -  14.4s
[CV] C=1 .............................................................
[CV] .................................... C=1, score=0.715448 -  14.4s
[CV] C=1 .............................................................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  7.2min finished


In [74]:
# Сравним результаты первых 20 - и предсказаний
print np.vstack([grid_all_features.best_estimator_.predict(X_scaled[:20]),y[:20]])

[[0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 1 0 1]
 [1 1 0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 1 1 1]]


In [83]:
grid_all_features.best_score_

0.71637808097540101

#### 2 Задание

In [9]:
cat_features =['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero',
                            'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero']

In [27]:
# Сотавляем только численные признаки
X_numeric = X.drop(cat_features, axis=1)

In [80]:
#Отмасштабируем
X_numeric_scaled = StandardScaler().fit_transform(X_numeric)

In [87]:
grid_num_search = GridSearchCV(log_reg,param_grid=C, scoring='roc_auc', cv=kf,verbose=5)

In [88]:
grid_numeric = grid_num_search.fit(X_numeric_scaled,y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] C=0.01 ..........................................................
[CV] ................................. C=0.01, score=0.716942 -  12.2s
[CV] C=0.01 ..........................................................
[CV] ................................. C=0.01, score=0.715495 -  11.9s
[CV] C=0.01 ..........................................................
[CV] ................................. C=0.01, score=0.717659 -  11.9s
[CV] C=0.01 ..........................................................
[CV] ................................. C=0.01, score=0.715779 -  12.5s
[CV] C=0.01 ..........................................................
[CV] ................................. C=0.01, score=0.716456 -  12.5s
[CV] C=0.03 ..........................................................
[CV] ................................. C=0.03, score=0.716960 -  12.9s
[CV] C=0.03 ..........................................................
[CV] ............

[Parallel(n_jobs=1)]: Done  17 tasks       | elapsed:  3.7min


[CV] .................................. C=0.3, score=0.717622 -  12.2s
[CV] C=0.3 ...........................................................
[CV] .................................. C=0.3, score=0.715670 -  12.7s
[CV] C=0.3 ...........................................................
[CV] .................................. C=0.3, score=0.716424 -  13.0s
[CV] C=1 .............................................................
[CV] .................................... C=1, score=0.716972 -  13.2s
[CV] C=1 .............................................................
[CV] .................................... C=1, score=0.715471 -  12.8s
[CV] C=1 .............................................................
[CV] .................................... C=1, score=0.717623 -  12.1s
[CV] C=1 .............................................................
[CV] .................................... C=1, score=0.715668 -  13.0s
[CV] C=1 .............................................................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  6.5min finished


In [89]:
grid_numeric.best_score_

0.7164659731972014

#### 4 Задание

In [6]:
# Колоныки с героями
heroes_columns = []
for i in xrange(5):
    heroes_columns.append('r%d_hero' % (i + 1))
    heroes_columns.append('d%d_hero' % (i + 1))

In [7]:
# Выдает
ids_of_heros = np.unique(features_data[heroes_columns])

In [102]:
print ids_of_heros.max(), ids_of_heros.size

112 108


#### 4 Задание

In [8]:
X_pick = np.zeros((features_data.shape[0], ids_of_heros.max()))

for i, match_id in enumerate(features_data.index):
    for p in xrange(5):
        X_pick[i, features_data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features_data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

#### 5 Задание

In [28]:
X_num_cat = np.hstack([X_numeric,X_pick])

In [29]:
X_num_cat_scaled = StandardScaler().fit_transform(X_num_cat)

In [30]:
grid_search =GridSearchCV(log_reg,param_grid=C, scoring='roc_auc', cv=kf,verbose=5)

In [31]:
grid_num_cat = grid_search.fit(X_num_cat_scaled,y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] C=0.01 ..........................................................
[CV] ................................. C=0.01, score=0.756448 -  31.3s
[CV] C=0.01 ..........................................................
[CV] ................................. C=0.01, score=0.749836 -  24.2s
[CV] C=0.01 ..........................................................
[CV] ................................. C=0.01, score=0.753499 -  25.1s
[CV] C=0.01 ..........................................................
[CV] ................................. C=0.01, score=0.751790 -  25.0s
[CV] C=0.01 ..........................................................
[CV] ................................. C=0.01, score=0.747576 -  23.5s
[CV] C=0.03 ..........................................................
[CV] ................................. C=0.03, score=0.756470 -  24.7s
[CV] C=0.03 ..........................................................
[CV] ............

[Parallel(n_jobs=1)]: Done  17 tasks       | elapsed:  7.3min


[CV] .................................. C=0.3, score=0.753365 -  24.5s
[CV] C=0.3 ...........................................................
[CV] .................................. C=0.3, score=0.751752 -  24.2s
[CV] C=0.3 ...........................................................
[CV] .................................. C=0.3, score=0.747608 -  25.6s
[CV] C=1 .............................................................
[CV] .................................... C=1, score=0.756480 -  25.3s
[CV] C=1 .............................................................
[CV] .................................... C=1, score=0.749781 -  25.3s
[CV] C=1 .............................................................
[CV] .................................... C=1, score=0.753361 -  24.8s
[CV] C=1 .............................................................
[CV] .................................... C=1, score=0.751750 -  24.3s
[CV] C=1 .............................................................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 12.8min finished


In [131]:
grid_num_cat.best_score_

0.75175812973604672

#### 6 Задание

In [32]:
features_test = pd.read_csv('features_test.csv',index_col='match_id')

In [44]:
X_test = features_test.drop(['lobby_type','r1_hero', 'r2_hero',  'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero','d3_hero','d4_hero', 'd5_hero'],axis=1)

In [38]:
X_pick = np.zeros((features_test.shape[0], 112))

for i, match_id in enumerate(features_test.index):
    for p in xrange(5):
        X_pick[i, features_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [45]:
X_test.fillna(0,inplace=True)

In [46]:
X_test = StandardScaler().fit_transform(X_test)

In [47]:
X_test = np.hstack([X_test,X_pick])

In [48]:
pred_y = grid_num_cat.best_estimator_.predict_proba(X_test)[:,1]

In [49]:
print np.min(pred_y), np.max(pred_y)

0.00662927420173 0.995364185517


### Отсчет по логистической регрессии

1. <b>0.716378</b> - лучшее качество. Видно, что логистическая регрессия обучаеться в разы быстрее, но качество не перезодит за 0.72. Если градиентный бустинг обучить на достаточном кол - ве алгоритмов, то качество будет выше даже на неотмасштабированных данных.
<br><br>
2. <b>0.716466</b> - качество без категориальных признаков немного улучшелось. Дело в том что линейная классификация не разделяет признаки и категориальные считает как числовые, не учитывая их значимости.
<br><br>
3. Судя по данным из выборки, количество возможных героев - <b>112</b>. В данной выборке использовалось только - <b>108</b>
<br><br>
4. <b>0.751758</b> - Качество при использовании мешка слов. Видно, что по сравнению с придедущими значениями, этот метод улучшает модель. Это обусловливается тем, что исользуется вектор категориальных значений
<br><br>
5. В тестовой выборке использовалась модель логистической регрессии:<br>
      мин - <b>0.00662927420173</b>; макс - <b>0.995364185517</b>